In [6]:
import numpy as np
import os 
import cv2
import random
import sys 
import csv
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
import bz2
from collections import defaultdict
from bz2 import BZ2File
from datetime import datetime
from PIL import Image
from sklearn.metrics import roc_curve, accuracy_score
from sklearn.metrics import auc, confusion_matrix
import sys
from scikit_posthocs import posthoc_ttest
sys.path.append('.')
from src.modules import *

In [101]:
for j in range(len((pred_results_all[f'attr{0}']))):
    print(len(pred_results_all[f'attr{3}'][j][0]))

4524
4524
4524
4524
4524
4524
4524
4524
4524
4524
4524
4524
4524
4524
4378
4376
4376
4375
4381
4374
4384
4385
4372


In [46]:
def statistics(results_all, bs_num=50):
    
    np.random.seed(10)
    
    perf_models = {}
    for j in range(len(results_all['attr0'])):
        perf_models[f'perf_bs{j}'] = {'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]}
    
    for k in range(bs_num):
        
#         num_sample = len(results_all[f'attr{i}'][j][0])
#         class0idx = np.where(results_all['attr3'][10][0]==0)[0]
#         class1idx = np.where(results_all['attr3'][10][0]==1)[0]
#         idx0 = np.random.choice(class0idx, len(class0idx))
#         idx1 = np.random.choice(class1idx, len(class1idx))
#         idx = list(idx0) + list(idx1)
        
        for i in range(7):
            
            class0idx = np.where(results_all[f'attr{i}'][10][0]==0)[0]
            class1idx = np.where(results_all[f'attr{i}'][10][0]==1)[0]
            idx0 = np.random.choice(class0idx, len(class0idx))
            idx1 = np.random.choice(class1idx, len(class1idx))
            idx = list(idx0) + list(idx1)
            
            for j in range(len(perf_models)):
                results_epoch = results_all[f'attr{i}'][j]
                
                if len(results_epoch[0]) < 4524:
                    class0idx = np.where(results_all[f'attr{i}'][j][0]==0)[0]
                    class1idx = np.where(results_all[f'attr{i}'][j][0]==1)[0]
                    idx0 = np.random.choice(class0idx, len(class0idx))
                    idx1 = np.random.choice(class1idx, len(class1idx))
                    idx = list(idx0) + list(idx1)
                
#                 idx = [p for p in idx if p < len(results_epoch[0])]
                _gt = results_epoch[0][idx]
                _pred = results_epoch[1][idx]
                _attr = results_epoch[2][:, idx]

                val_es_acc, val_es_auc, val_aucs_by_attrs, val_dpds, val_eods, val_between_group_disparity = evalute_comprehensive_perf(_pred,
                                                                                                                                    _gt,
                                                                                                                                    _attr)

                perfi = [val_es_acc[i], val_es_auc[i]]
                perfi.extend(val_aucs_by_attrs[i])
                perfi.append(val_dpds[i])
                perfi.append(val_eods[i])
                perfi.extend(val_between_group_disparity[i])
#                 perf_bs_proposed[f'attr{i}'].append(perfi)

                perf_models[f'perf_bs{j}'][f'attr{i}'].append(perfi)
    
    attr_pvalues = {'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]}
    
    for i in range(7):
        num_metrics = len(perf_models['perf_bs0'][f'attr{i}'][0])
        for k in range(num_metrics):
            vpaired = []
            for j in range(len(perf_models.keys())):
                vpaired.append(np.array(perf_models[f'perf_bs{j}'][f'attr{i}'])[:, k])
            
            vpaired = np.array(vpaired)
            vpaired = vpaired[:, ~np.isnan(vpaired).any(axis=0)]
            
            pvalues = posthoc_ttest(vpaired)
            
            attr_pvalues[f'attr{i}'].append(pvalues)
            
    return attr_pvalues

In [8]:
# # transform adversarial results

folder = '/data/home/shim/pyspace/others/pyspace/ICLR_30k/Harvard-DR30k_results/results_colorfundus_FINAL/dr_slo_fundus_adversarial'

attrs = ['race', 'gender', 'ethnicity', 'maritalstatus', 'language', 'gender_race', 'gender_ethnicity']
attrmaps = {'attr0':'Race', 'attr1':'Gender', 'attr2':'Ethnicity', 'attr3':'Maritalstatus', 'attr4':'Language', 
                'attr5':'Gender+Race', 'attr6':'Gender+Ethnicity'}


outcome_adv = {'vgg':{'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]},
              'efficientnet':{'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]},
              'resnet':{'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]},
              'swin':{'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]},
              'convnext':{'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]},
              'densenet':{'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]},
              'ViT-B':{'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]}}

for idx, attr in enumerate(attrs):
    modality = f'dr_slo_fundus_{attr}'

    modality_folder = f'{folder}/{modality}'

    bestresults = []
    epochresults = []
    for f in os.listdir(modality_folder):
        if f.endswith('.csv'):
            paths = pd.read_csv(os.path.join(modality_folder, f))['  path']
            bestresults.append([f, paths[0].split('/')[-1].split('auc')[0]])
        else:
            epochresults.append(f)
    pairedresults = {}
    for i, v in enumerate(bestresults):
        for p in epochresults:
            if v[1] in p:
                pairedresults[i] = [v[0], p]
    
    for k in pairedresults.keys():
        v = pairedresults[k]
    
        resultpd = pd.read_csv(os.path.join(modality_folder, v[0]))
        results = {}
        attrgroupnums = {}
        for name, data in resultpd.items():
            results[name.strip()] = data.values[-1]
            if 'group' in name:
                attri = name.split('_')[1]
                if attri not in attrgroupnums.keys():
                    attrgroupnums[attri] = 1
                else:
                    attrgroupnums[attri] += 1
        modelname = v[0].split('_')[1]
        
        
        vs = [results[f'esacc_attr{idx}'], results['acc'], results[f'esauc_attr{idx}'], results['auc']]
        vs = vs + [results[f'auc_attr{idx}_group{j}'] for j in range(attrgroupnums[f'attr{idx}'])]
        vs = vs + [results[f'dpd_attr{idx}'], results[f'eod_attr{idx}'], 
                   results[f'std_group_disparity_attr{idx}'],
                   results[f'max_group_disparity_attr{idx}']]
        
        npzdata = np.load(os.path.join(modality_folder, v[1], 'pred_gt_best_epoch.npz'))
        test_gt = npzdata['test_gt']
        test_pred = npzdata['test_pred']
        test_attr = npzdata['test_attr']
        vs_epoch = [test_gt, test_pred, test_attr]
        
#         print(len(test_gt), len(test_pred), len(test_attr), os.path.join(modality_folder, v[1], 'pred_gt_best_epoch.npz'))
       
        outcome_adv[modelname][f'attr{idx}'] = (vs, vs_epoch)
    

In [18]:
# # transform proposed model results

folder = '/data/home/shim/pyspace/others/pyspace/ICLR_30k/Harvard-DR30k_results/results_colorfundus_FINAL/dr_slo_fundus_fis'

attrs = ['race', 'gender', 'ethnicity', 'maritalstatus', 'language', 'genderrace', 'genderethnicity']
attrmaps = {'attr0':'Race', 'attr1':'Gender', 'attr2':'Ethnicity', 'attr3':'Maritalstatus', 'attr4':'Language', 
                'attr5':'Gender+Race', 'attr6':'Gender+Ethnicity'}


outcome_fis = {'efficientnet':{'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]},
              'ViT-B':{'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]}}

for idx, attr in enumerate(attrs):
    modality = f'dr_slo_fundus_{attr}_fis'

    modality_folder = f'{folder}/{modality}'

    bestresults = []
    epochresults = []
    for f in os.listdir(modality_folder):
        if f.endswith('.csv'):
            paths = pd.read_csv(os.path.join(modality_folder, f))['  path']
            bestresults.append([f, paths[0].split('/')[-1].split('auc')[0]])
        else:
            epochresults.append(f)
    pairedresults = {}
    for i, v in enumerate(bestresults):
        for p in epochresults:
            if v[1] in p:
                pairedresults[i] = [v[0], p]
    
    for k in pairedresults.keys():
        v = pairedresults[k]
    
        resultpd = pd.read_csv(os.path.join(modality_folder, v[0]))
        results = {}
        attrgroupnums = {}
        for name, data in resultpd.items():
            results[name.strip()] = data.values[-1]
            if 'group' in name:
                attri = name.split('_')[1]
                if attri not in attrgroupnums.keys():
                    attrgroupnums[attri] = 1
                else:
                    attrgroupnums[attri] += 1
        modelname = v[0].split('_')[1]
        
        
        vs = [results[f'esacc_attr{idx}'], results['acc'], results[f'esauc_attr{idx}'], results['auc']]
        vs = vs + [results[f'auc_attr{idx}_group{j}'] for j in range(attrgroupnums[f'attr{idx}'])]
        vs = vs + [results[f'dpd_attr{idx}'], results[f'eod_attr{idx}'], 
                   results[f'std_group_disparity_attr{idx}'],
                   results[f'max_group_disparity_attr{idx}']]
        
        npzdata = np.load(os.path.join(modality_folder, v[1], 'pred_gt_best_epoch.npz'))
        test_gt = npzdata['test_gt']
        test_pred = npzdata['test_pred']
        test_attr = npzdata['test_attr']
        vs_epoch = [test_gt, test_pred, test_attr]
        
#         print(len(test_gt), len(test_pred), len(test_attr), os.path.join(modality_folder, v[1], 'pred_gt_best_epoch.npz'))
       
        outcome_fis[modelname][f'attr{idx}'] = (vs, vs_epoch)
    

In [73]:
# statistic analysis
modality = 'slo_fundus' #'slo_fundus or oct_bscans'
folder = '/data/home/shim/pyspace/others/pyspace/ICLR_30k/Harvard-DR30k_results/results_colorfundus_FINAL'


modality_folder = f'{folder}/dr_{modality}_race'

model_order = ['vgg', 'swin', 'resnet', 'convnext', 'densenet', 'efficientnet', 'ViT-B', 
               'vgg_oversample','swin_oversample', 'resnet_oversample', 'convnext_oversample', 
               'densenet_oversample', 'efficientnet_oversample', 'ViT-B_oversample']

attrmaps = {'attr0':'Race', 'attr1':'Gender', 'attr2':'Ethnicity', 'attr3':'Maritalstatus', 'attr4':'Language', 
            'attr5':'Gender+Race', 'attr6':'Gender+Ethnicity'}

bestresults = []
epochresults = []
for f in os.listdir(modality_folder):
    if f.endswith('.csv'):
        print(f)
        paths = pd.read_csv(os.path.join(modality_folder, f))['  path']
        bestresults.append([f, paths[0].split('/')[-1].split('auc')[0]])
    else:
        epochresults.append(f)
        
pairedresults = {}
modelindex = {}
for i, v in enumerate(bestresults):
    for p in epochresults:
        if v[1] in p:
            pairedresults[i] = [v[0], p]
            modelname = v[0].split('_')[1]
            if 'oversample' in v[0]:
                modelname = modelname + '_oversample'
            modelindex[modelname] = i

best_results_all = {'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]}
column_names = {'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]}
rownames = []
pred_results_all = {'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]}

num_attrs = 7

for k in model_order:
    if k not in modelindex.keys():
        continue
    k = modelindex[k]
    v = pairedresults[k]
    resultpd = pd.read_csv(os.path.join(modality_folder, v[0]))
    results = {}
    attrgroupnums = {}
    for name, data in resultpd.items():
        results[name.strip()] = data.values[-1]
        if 'group' in name:
            attri = name.split('_')[1]
            if attri not in attrgroupnums.keys():
                attrgroupnums[attri] = 1
            else:
                attrgroupnums[attri] += 1
    modelname = v[0].split('_')[1]
    if 'oversample' in v[0]:
        modelname = modelname + '_oversample'
    
    rownames.append(modelname)
    print(modelname)
    npzdata = np.load(os.path.join(modality_folder, v[1], 'pred_gt_best_epoch.npz'))
    test_gt = npzdata['test_gt']
    test_pred = npzdata['test_pred']
    test_attr = npzdata['test_attr']
    
    for i in range(num_attrs):
        vs = [results[f'esacc_attr{i}'], results['acc'], results[f'esauc_attr{i}'], results['auc']]
        vs = vs + [results[f'auc_attr{i}_group{j}'] for j in range(attrgroupnums[f'attr{i}'])]
        vs = vs + [results[f'dpd_attr{i}'], results[f'eod_attr{i}'], 
                   results[f'std_group_disparity_attr{i}'],
                   results[f'max_group_disparity_attr{i}']]
        best_results_all[f'attr{i}'].append(vs)
        
        vsname = ['esacc', 'acc', 'esauc', 'auc']
        vsname = vsname + [f'auc_group{j}' for j in range(attrgroupnums[f'attr{i}'])]
        vsname = vsname + ['dpd', 'eod', 'std_group_disparity', 'max_group_disparity']
        column_names[f'attr{i}'] = vsname

        pred_results_all[f'attr{i}'].append([test_gt, test_pred, test_attr])


# append adversarial outcome
modeladv_order = ['vgg', 'swin', 'resnet', 'convnext', 'densenet', 'efficientnet', 'ViT-B']
for modelname in modeladv_order:
    modelname_subfix = f'{modelname}_adv'
    rownames.append(modelname_subfix)
    print(modelname_subfix)
    for attri in outcome_adv[modelname].keys():
#         try:
        best_results_all[attri].append(outcome_adv[modelname][attri][0])
        pred_results_all[attri].append(outcome_adv[modelname][attri][1])
#         except: 
#             pass


# append proposed model outcome
modelfis_order = ['efficientnet', 'ViT-B']
for modelname in modelfis_order:
    modelname_subfix = f'{modelname}_fis'
    rownames.append(modelname_subfix)
    print(modelname_subfix)
    for attri in outcome_fis[modelname].keys():
#         try:
        best_results_all[attri].append(outcome_fis[modelname][attri][0])
        pred_results_all[attri].append(outcome_fis[modelname][attri][1])
#         except: 
#             pass
    
# write best outcome    
writer = pd.ExcelWriter(f"{folder}/dr_{modality}_race_combined_outcome_FINAL.xlsx", engine="xlsxwriter")

for k in attrmaps.keys():
    attridx = int(k[-1])
    pdattri = pd.DataFrame(best_results_all[f'attr{attridx}'], 
                 index=rownames, 
                 columns=column_names[f'attr{attridx}'])
    pdattri.to_excel(writer, sheet_name=attrmaps[k])

    
attr_pvalues = statistics(pred_results_all)

for i in range(num_attrs):
    rowidx = 0
    vsname = ['esacc', 'acc', 'esauc', 'auc']
    vsname = vsname + [f'auc_group{j}' for j in range(len(attr_pvalues[f'attr{i}']))]
    vsname = vsname + ['dpd', 'eod', 'std_group_disparity', 'max_group_disparity']
    
    for idx, pvalues in enumerate(attr_pvalues[f'attr{i}']):
        
        rownames_prefix = [str(v)+'@'+vsname[idx] for v in rownames]
        
        pvdf = pd.DataFrame(np.array(pvalues), index=rownames_prefix, columns=rownames_prefix)
        
        pvdf.to_excel(writer, sheet_name=attrmaps[f'attr{i}']+'_statistics', startrow=rowidx)
        rowidx += len(pvalues) + 2
    
writer.close()
# pairedresults

best_ViT-B_slo_fundus_race.csv
best_swin_slo_fundus_race.csv
best_resnet_slo_fundus_race_oversample.csv
best_ViT-B_slo_fundus_race_oversample.csv
best_convnext_slo_fundus_race.csv
best_vgg_slo_fundus_race.csv
best_swin_slo_fundus_race_oversample.csv
best_efficientnet_slo_fundus_race.csv
best_efficientnet_slo_fundus_race_oversample.csv
best_vgg_slo_fundus_race_oversample.csv
best_densenet_slo_fundus_race_oversample.csv
best_resnet_slo_fundus_race.csv
best_convnext_slo_fundus_race_oversample.csv
best_densenet_slo_fundus_race.csv
vgg
swin
resnet
convnext
densenet
efficientnet
ViT-B
vgg_oversample
swin_oversample
resnet_oversample
convnext_oversample
densenet_oversample
efficientnet_oversample
ViT-B_oversample
vgg_adv
swin_adv
resnet_adv
convnext_adv
densenet_adv
efficientnet_adv
ViT-B_adv
efficientnet_fis
ViT-B_fis


/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

...

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

ValueError: Cannot infer input format.
Please specify `val_id` and `group_id` args

In [103]:
results_all = pred_results_all

np.random.seed(10)
bs_num = 50    
perf_models = {}
for j in range(len(results_all['attr0'])):
    perf_models[f'perf_bs{j}'] = {'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]}

for k in range(bs_num):

#         num_sample = len(results_all[f'attr{i}'][j][0])
#         class0idx = np.where(results_all['attr3'][10][0]==0)[0]
#         class1idx = np.where(results_all['attr3'][10][0]==1)[0]
#         idx0 = np.random.choice(class0idx, len(class0idx))
#         idx1 = np.random.choice(class1idx, len(class1idx))
#         idx = list(idx0) + list(idx1)

    for i in range(7):

        class0idx = np.where(results_all[f'attr{i}'][10][0]==0)[0]
        class1idx = np.where(results_all[f'attr{i}'][10][0]==1)[0]
        idx0 = np.random.choice(class0idx, len(class0idx))
        idx1 = np.random.choice(class1idx, len(class1idx))
        idx = list(idx0) + list(idx1)

        for j in range(len(perf_models)):
            results_epoch = results_all[f'attr{i}'][j]

            if len(results_epoch[0]) < 4524:
                class0idx = np.where(results_all[f'attr{i}'][j][0]==0)[0]
                class1idx = np.where(results_all[f'attr{i}'][j][0]==1)[0]
                idx0 = np.random.choice(class0idx, len(class0idx))
                idx1 = np.random.choice(class1idx, len(class1idx))
                idx = list(idx0) + list(idx1)

            _gt = results_epoch[0][idx]
            _pred = results_epoch[1][idx]
            _attr = results_epoch[2][:, idx]

            val_es_acc, val_es_auc, val_aucs_by_attrs, val_dpds, val_eods, val_between_group_disparity = evalute_comprehensive_perf(_pred,
                                                                                                                                _gt,
                                                                                                                                _attr)

            perfi = [val_es_acc[i], val_es_auc[i]]
            perfi.extend(val_aucs_by_attrs[i])
            perfi.append(val_dpds[i])
            perfi.append(val_eods[i])
            perfi.extend(val_between_group_disparity[i])
#                 perf_bs_proposed[f'attr{i}'].append(perfi)

            perf_models[f'perf_bs{j}'][f'attr{i}'].append(perfi)

attr_pvalues = {'attr0':[], 'attr1':[], 'attr2':[], 'attr3':[], 'attr4':[], 'attr5':[], 'attr6':[]}

for i in range(7):
    num_metrics = len(perf_models['perf_bs0'][f'attr{i}'][0])
    for k in range(num_metrics):
        vpaired = []
        for j in range(len(perf_models.keys())):
            vpaired.append(np.array(perf_models[f'perf_bs{j}'][f'attr{i}'])[:, k])

        vpaired = np.array(vpaired)
        vpaired = vpaired[:, ~np.isnan(vpaired).any(axis=0)]
        
        print(vpaired)
        
        pvalues = posthoc_ttest(vpaired)

        attr_pvalues[f'attr{i}'].append(pvalues)



/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

...

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/data/home/shim/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1029: UndefinedMetricWarning: No positive sam

[[0.89892354 0.86963446 0.87254384 ... 0.87655145 0.90257335 0.8709249 ]
 [0.89051142 0.89169942 0.89438124 ... 0.88740884 0.89177422 0.89234345]
 [0.87516454 0.87581716 0.85622481 ... 0.8666692  0.86121412 0.87219146]
 ...
 [0.90438557 0.91225878 0.89409845 ... 0.89542171 0.91269912 0.89056369]
 [0.89394171 0.8810391  0.89492271 ... 0.90175683 0.90097784 0.90762944]
 [0.88159934 0.88523562 0.87105594 ... 0.88779864 0.87414814 0.87102782]]
[[0.55337052 0.5149806  0.49864525 ... 0.51249704 0.5427351  0.4982329 ]
 [0.86429134 0.78499004 0.80213702 ... 0.83252151 0.80562872 0.841069  ]
 [0.56979164 0.54512222 0.5976909  ... 0.70439163 0.62489154 0.63872355]
 ...
 [0.82944619 0.86807584 0.87438885 ... 0.81506021 0.84173057 0.86830578]
 [0.8114495  0.74444633 0.69487805 ... 0.84632982 0.77189695 0.83827768]
 [0.74414268 0.83099888 0.84669979 ... 0.87789414 0.84473082 0.78148093]]
[[0.61624892 0.58124098 0.47878788 ... 0.565625   0.71230159 0.78929228]
 [0.90458015 0.95229941 0.92592593 ... 

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.01168622 0.05204884 0.02877953 ... 0.03691561 0.02943973 0.00755576]
 [0.0121337  0.01084518 0.0147228  ... 0.00956118 0.01143641 0.01102111]
 ...
 [0.03736966 0.01427184 0.00022836 ... 0.02208976 0.02642075 0.02299731]
 [0.01970669 0.01295341 0.00804763 ... 0.00203989 0.01087492 0.01915185]
 [0.00260669 0.00970218 0.01012278 ... 0.00546661 0.01362739 0.02110669]]
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.0433305  0.34721161 0.04772727 ... 0.15679443 0.18775064 0.09865005]
 [0.23740511 0.19731183 0.17961165 ... 0.18028846 0.18032787 0.22137681]
 ...
 [0.0768978  0.21339713 0.18927973 ... 0.01977401 0.06880734 0.07827715]
 [0.30729167 0.31527094 0.24352332 ... 0.33149171 0.23902439 0.28645833]
 [0.13085106 0.19912609 0.0179402  ... 0.05539906 0.1975945  0.05813953]]
[[0.17994926 0.20811969 0.13815711 ... 0.15624589 0.03422784 0.17504342]
 [0.02989203 0.00859691 0.03176113 ... 

[[0.45537077 0.40914802 0.35602789 ... 0.37921831 0.39971494 0.35773496]
 [0.7150784  0.64166506 0.66778125 ... 0.75004876 0.74320107 0.68014408]
 [0.62241083 0.55500774 0.56193849 ... 0.50793375 0.48051616 0.50417079]
 ...
 [0.67277956 0.69063261 0.81206021 ... 0.68836811 0.80135305 0.76018624]
 [0.66479051 0.62780273 0.65672673 ... 0.62311005 0.59400683 0.71422046]
 [0.76509943 0.76902408 0.70116556 ... 0.72591675 0.74617924 0.74542857]]
[[0.82299742 0.83610188 0.81666667 ... 0.73062016 0.85454545 0.74275362]
 [0.90405117 0.88627049 0.95       ... 0.94140625 0.93439153 0.93080054]
 [0.67430025 0.51640212 0.57304965 ... 0.66467958 0.66533467 0.58019217]
 ...
 [0.78481013 0.68987342 0.92735043 ... 0.875      0.91428571 0.91952984]
 [0.9894958  0.96690307 0.98133117 ... 0.93719807 0.97235772 0.9825784 ]
 [0.93846154 0.88682432 0.95072993 ... 0.86044521 0.80136986 0.87819549]]
[[0.7421875  0.63356164 0.59591837 ... 0.66428571 0.56910569 0.52328767]
 [0.87410072 0.82414698 0.80885312 ... 

In [105]:
attr_pvalues

{'attr0': [              1             2             3             4             5   \
  1   1.000000e+00  1.706516e-02  1.282713e-01  5.074582e-03  8.622065e-05   
  2   1.706516e-02  1.000000e+00  7.798037e-05  8.350462e-08  9.290133e-02   
  3   1.282713e-01  7.798037e-05  1.000000e+00  2.133272e-01  4.586063e-08   
  4   5.074582e-03  8.350462e-08  2.133272e-01  1.000000e+00  4.185786e-12   
  5   8.622065e-05  9.290133e-02  4.586063e-08  4.185786e-12  1.000000e+00   
  6   6.300808e-07  2.112862e-03  2.037887e-10  1.518390e-14  9.878486e-02   
  7   4.732374e-12  4.428619e-08  5.237882e-16  9.648713e-21  1.093279e-05   
  8   1.456126e-01  1.744185e-05  7.745476e-01  7.850127e-02  9.361016e-10   
  9   4.084484e-10  1.410698e-06  1.993115e-13  4.157806e-17  1.445164e-04   
  10  3.644057e-28  1.138387e-35  2.450249e-25  9.619994e-25  1.116167e-40   
  11  8.989129e-05  6.052188e-02  1.034905e-07  3.695356e-11  6.849214e-01   
  12  1.084786e-03  3.290846e-01  1.544962e-06  4.78832

In [106]:
# write best outcome    
writer = pd.ExcelWriter(f"{folder}/dr_{modality}_race_combined_outcome_FINAL.xlsx", engine="xlsxwriter")

for k in attrmaps.keys():
    attridx = int(k[-1])
    pdattri = pd.DataFrame(best_results_all[f'attr{attridx}'], 
                 index=rownames, 
                 columns=column_names[f'attr{attridx}'])
    pdattri.to_excel(writer, sheet_name=attrmaps[k])

for i in range(num_attrs):
    rowidx = 0
    vsname = ['esacc', 'acc', 'esauc', 'auc']
    vsname = vsname + [f'auc_group{j}' for j in range(len(attr_pvalues[f'attr{i}']))]
    vsname = vsname + ['dpd', 'eod', 'std_group_disparity', 'max_group_disparity']
    
    for idx, pvalues in enumerate(attr_pvalues[f'attr{i}']):
        
        rownames_prefix = [str(v)+'@'+vsname[idx] for v in rownames]
        
        pvdf = pd.DataFrame(np.array(pvalues), index=rownames_prefix, columns=rownames_prefix)
        
        pvdf.to_excel(writer, sheet_name=attrmaps[f'attr{i}']+'_statistics', startrow=rowidx)
        rowidx += len(pvalues) + 2
    
writer.close()

ValueError: Empty data passed with indices specified.

In [59]:
vpaired

array([[0.37282387, 0.37350942],
       [0.68487511, 0.68776765],
       [0.47875738, 0.69302492],
       [0.61825751, 0.59030093],
       [0.69174002, 0.5401925 ],
       [0.58353078, 0.63335976],
       [0.58037293, 0.75544467],
       [0.43504317, 0.44702688],
       [0.6979484 , 0.67281025],
       [0.68675944, 0.51180155],
       [0.63189138, 0.61659574],
       [0.63222122, 0.60754523],
       [0.61224325, 0.5127601 ],
       [0.61544085, 0.56301661],
       [0.47757747, 0.49160604],
       [0.84199415, 0.72929505],
       [0.73558346, 0.66174624],
       [0.55795518, 0.52401919],
       [0.74121749, 0.64901251],
       [0.78977296, 0.62690298],
       [0.82172192, 0.6896577 ],
       [0.74108591, 0.69961788],
       [0.680122  , 0.81454344]])

In [ ]:
# for i in range(num_attrs):
#     rowidx = 0
#     vsname = ['esacc', 'acc', 'esauc', 'auc']
#     vsname = vsname + [f'auc_group{j}' for j in range(len(attr_pvalues[f'attr{i}']))]
#     vsname = vsname + ['dpd', 'eod', 'std_group_disparity', 'max_group_disparity']
    
#     for idx, pvalues in enumerate(attr_pvalues[f'attr{i}']):
        
#         rownames_prefix = [str(v)+'@'+vsname[idx] for v in rownames]
        
#         pvdf = pd.DataFrame(np.array(pvalues), index=rownames_prefix, columns=rownames_prefix)
        
#         pvdf.to_excel(writer, sheet_name=attrmaps[f'attr{i}']+'_statistics', startrow=rowidx)
#         rowidx += len(pvalues) + 2
    
# writer.close()

In [14]:
attridx = 0
pd.DataFrame(best_results_all[f'attr{attridx}'], index=rownames, columns=column_names[f'attr{attridx}'])

,esacc,acc,esauc,auc,auc_group0,auc_group1,auc_group2,dpd,eod,std_group_disparity,max_group_disparity
vgg,0.7796,0.8520,0.6507,0.7168,0.6336,0.7337,0.7153,0.0691,0.1648,0.0607,0.1396
swin,0.8722,0.9247,0.6572,0.8170,0.6457,0.7655,0.8375,0.0187,0.2224,0.0968,0.2347
resnet,0.8549,0.9137,0.6525,0.7505,0.6411,0.7214,0.7622,0.0009,0.0108,0.0670,0.1614
convnext,0.7806,0.8910,0.6317,0.7915,0.6221,0.7302,0.8137,0.0474,0.2157,0.0991,0.2420
densenet,0.8444,0.9120,0.6427,0.7769,0.6094,0.7487,0.7901,0.0255,0.1472,0.0995,0.2325
efficientnet,0.8716,0.9217,0.6713,0.7939,0.7272,0.7002,0.8162,0.0020,0.1687,0.0624,0.1461
ViT-B,0.8486,0.9223,0.7652,0.8570,0.8065,0.8004,0.8698,0.0099,0.2708,0.0366,0.0810
vgg_oversample,0.6566,0.7473,0.6345,0.7082,0.6002,0.7114,0.7131,0.0956,0.1218,0.0746,0.1593
swin_oversample,0.8569,0.9190,0.6287,0.7810,0.6501,0.6982,0.8095,0.0118,0.2473,0.0855,0.2042
resnet_oversample,0.8545,0.9130,0.6659,0.7305,0.6639,0.7085,0.7390,0.0000,0.0000,0.0422,0.1028
